<h1><center> Analyzing CIA Factbook Data Using SQLite <center/><h1/>

In this project I will be analyzing CIA Factbook Data to extract insights (demographic and otherwise) on countries aroud the globe. I will perform my analysis using SQLite.

Let's begin by ensuring jupyter can run sql commands

In [1]:
!conda install -yc conda-forge ipython-sql

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\ibibe\Anaconda3

  added / updated specs:
    - ipython-sql


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.12.5  |       h5b45459_0         173 KB  conda-forge
    certifi-2020.12.5          |   py37h03978a9_1         143 KB  conda-forge
    conda-4.9.2                |   py37h03978a9_0         3.0 MB  conda-forge
    importlib-metadata-3.7.0   |   py37h03978a9_0          24 KB  conda-forge
    ipython-sql-0.3.9          |py37hc8dfbb8_1002          28 KB  conda-forge
    openssl-1.1.1j             |       h8ffe710_0         5.8 MB  conda-forge
    prettytable-2.1.0          |     pyhd8ed1ab_0          23 KB  conda-forge
    sqlparse-0.4.1             |     pyh9f0ad1d_0          34 KB  conda-forge
    typing_extensions-3.7.4.3  |             py_0          25 KB  conda-forg

Now let's connect to the database file itself

In [2]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

In some cases, we may not know what tables are present in our database. So best practice would be to actually run a command that displays the tables present in your database. Let's test that out now

In [3]:
%%sql
SELECT *
FROM sqlite_master
WHERE type='table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


SQLite databases have some system tables which contain useful information about the database. These tables begin with the `sqlite_`prefix. One of such tables is the `sqlite_master` table which is a master listing of all database objects in the database and the SQL used to create each object. We use that to identify the tables available in our database as demonstrated above. 

Our query shows there's two tables; `sqlite_sequence` and `facts`. `facts` is our table of interest. Let's query it to be sure of this.

In [4]:
%%sql
SELECT *
FROM facts
LIMIT 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


### Exploratory Data Analysis

Let's explore the dataset. Its possible there might be outliers present so running summary statistics will give us an idea of what values to look out for. 

In [5]:
%%sql

SELECT 
       MIN(population) AS min_population,
       MAX(population) AS max_population,
       MIN(population_growth) AS min_population_growth,
       MAX(population_growth) AS max_population_growth
FROM facts;

 * sqlite:///factbook.db
Done.


min_population,max_population,min_population_growth,max_population_growth
0,7256490011,0.0,4.02


Right away we can see there's a country with a min_population of 0 which may be a red flag. In addition, there's a country with about 7.2 billion people which is very similar to the entire population of earth. Let's find out which countries these are. 

In [6]:
%%sql
-- Minimum Population

SELECT *
FROM facts
WHERE population =    (
    SELECT MIN(population)
FROM facts);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


In [7]:
%%sql
-- Maximum Population

SELECT *
FROM facts
WHERE population =    (
    SELECT MAX(population)
FROM facts);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


My hypothesis regarding the maximum population was correct; Indeed the data entry with this value corresponds to a country known as World which is essentially the world population. Antartica is technically a continent, not a country. So it explains the lack of a population.

With those obvious irregularities known, i will recalculate the summary statistics without including them

In [8]:
%%sql

SELECT 
       MIN(population) AS min_population,
       MAX(population) AS max_population,
       MIN(population_growth) AS min_population_growth,
       MAX(population_growth) AS max_population_growth
FROM facts
WHERE name NOT IN ('Antarctica', 'World');

 * sqlite:///factbook.db
Done.


min_population,max_population,min_population_growth,max_population_growth
48,1367485388,0.0,4.02


We can see that the values have changed. Let's double check to see if our exclusion did the trick

In [9]:
%%sql
-- Minimum Population

SELECT *
FROM facts
WHERE population =    (
    SELECT MIN(population)
FROM facts
WHERE name NOT IN ('Antarctica', 'World'));

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
238,pc,Pitcairn Islands,47,47,0,48,0.0,None,None,None


In [10]:
%%sql
-- Maximum Population

SELECT *
FROM facts
WHERE population =    (
    SELECT MAX(population)
FROM facts
WHERE name NOT IN ('Antarctica', 'World'));

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
37,ch,China,9596960,9326410,270550,1367485388,0.45,12.49,7.53,0.44


The Pitcairn Islands and China have the lowert and highest populations respectively. With that taken care of, I can move on and derive some interesting insights:

1. Which counties have above average populations
2. Which countries have below average land areas
3. Which counties are experiencing brain drain (high migration rate)

Let's start with the population

In [11]:
%%sql
SELECT *
FROM facts
WHERE population >
    (SELECT AVG(population)
            FROM facts
            WHERE name NOT IN ('Antarctica', 'World'))
AND name NOT IN ('Antarctica', 'World')
ORDER BY population DESC
LIMIT 10;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
37,ch,China,9596960,9326410,270550,1367485388,0.45,12.49,7.53,0.44
77,in,India,3287263,2973193,314070,1251695584,1.22,19.55,7.32,0.04
197,ee,European Union,4324782,None,None,513949445,0.25,10.2,10.2,2.5
186,us,United States,9826675,9161966,664709,321368864,0.78,12.49,8.15,3.86
78,id,Indonesia,1904569,1811569,93000,255993674,0.92,16.72,6.37,1.16
24,br,Brazil,8515770,8358140,157630,204259812,0.77,14.46,6.58,0.14
132,pk,Pakistan,796095,770875,25220,199085847,1.46,22.58,6.49,1.54
129,ni,Nigeria,923768,910768,13000,181562056,2.45,37.64,12.9,0.22
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
143,rs,Russia,17098242,16377742,720500,142423773,0.04,11.6,13.69,1.69


Notice we have `European Union` on our list, but that is also not a country. Its is a union between more than 20 European countries. Let's include that in our filter 

In [12]:
%%sql
SELECT *
FROM facts
WHERE population >
    (
        SELECT AVG(population)
            FROM facts
            WHERE name NOT IN ('European Union', 'Antarctica', 'World')
    )
AND name NOT IN ('European Union','Antarctica', 'World')
ORDER BY population DESC
LIMIT 10;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
37,ch,China,9596960,9326410,270550,1367485388,0.45,12.49,7.53,0.44
77,in,India,3287263,2973193,314070,1251695584,1.22,19.55,7.32,0.04
186,us,United States,9826675,9161966,664709,321368864,0.78,12.49,8.15,3.86
78,id,Indonesia,1904569,1811569,93000,255993674,0.92,16.72,6.37,1.16
24,br,Brazil,8515770,8358140,157630,204259812,0.77,14.46,6.58,0.14
132,pk,Pakistan,796095,770875,25220,199085847,1.46,22.58,6.49,1.54
129,ni,Nigeria,923768,910768,13000,181562056,2.45,37.64,12.9,0.22
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
143,rs,Russia,17098242,16377742,720500,142423773,0.04,11.6,13.69,1.69
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0


The majority of the top 10 populated countries were in the Asian region, with a single representative from North America, Europe, Africa and South America present. One question one might ask is how much land area do these countries have. Let's find the top 10 countries with below-average land areas. 

In [13]:
%%sql
SELECT *
FROM facts
WHERE area_land <
    (
        SELECT AVG(area_land)
            FROM facts
            WHERE name NOT IN ('European Union', 'Antarctica', 'World')
    )
AND name NOT IN ('European Union','Antarctica', 'World')
ORDER BY area_land
LIMIT 10;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
190,vt,Holy See (Vatican City),0,0,0,842,0.0,None,None,None
117,mn,Monaco,2,2,0,30535,0.12,6.65,9.24,3.83
201,cr,Coral Sea Islands,3,3,0,None,None,None,None,None
198,at,Ashmore and Cartier Islands,5,5,0,None,None,None,None,None
244,bq,Navassa Island,5,5,0,None,None,None,None,None
253,pg,Spratly Islands,5,5,0,None,None,None,None,None
208,ip,Clipperton Island,6,6,0,None,None,None,None,None
233,gi,Gibraltar,6,6,0,29258,0.24,14.08,8.37,3.28
248,wq,Wake Island,6,6,0,None,None,None,None,None
252,pf,Paracel Islands,7,7,0,None,None,None,None,None


None of the top populated countries appear on our below-average list of area. So this metric isn't too insightful. Instead, i'll look at the population density instead.

In [14]:
%%sql
SELECT *, 
        CAST(population AS FLOAT)/CAST(area_land AS FLOAT) AS population_density
FROM facts
WHERE name NOT IN ('European Union','Antarctica', 'World')
ORDER BY population_density DESC
LIMIT 10;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,population_density
205,mc,Macau,28,28,0,592731,0.8,8.88,4.22,3.37,21168.964285714286
117,mn,Monaco,2,2,0,30535,0.12,6.65,9.24,3.83,15267.5
156,sn,Singapore,697,687,10,5674472,1.89,8.27,3.43,14.05,8259.784570596797
204,hk,Hong Kong,1108,1073,35,7141106,0.38,9.23,7.07,1.68,6655.27120223672
251,gz,Gaza Strip,360,360,0,1869055,2.81,31.11,3.04,0.0,5191.819444444444
233,gi,Gibraltar,6,6,0,29258,0.24,14.08,8.37,3.28,4876.333333333333
13,ba,Bahrain,760,760,0,1346613,2.41,13.66,2.69,13.09,1771.8592105263158
108,mv,Maldives,298,298,0,393253,0.08,15.75,3.89,12.68,1319.6409395973155
110,mt,Malta,316,316,0,413965,0.31,10.18,9.09,1.98,1310.01582278481
227,bd,Bermuda,54,54,0,70196,0.5,11.33,8.23,1.88,1299.9259259259259


Again none of our top 10 countries showed up in this list, but the countries here essentially have a lot of people within a small area. Could this be a problem in the future? the population growth column can give us some insights. We would expect countries with already high population densities to have issues in the future if population growth is high. We can see if the impact is negative by comparing the ratio of death_rate to birth_rate

In [15]:
%%sql
SELECT *, death_rate/birth_rate AS death_birth_ratio
FROM facts
WHERE population_growth >
    (
        SELECT AVG(population_growth)
            FROM facts
            WHERE name NOT IN ('European Union', 'Antarctica', 'World')
    )
AND name NOT IN ('European Union','Antarctica', 'World')
ORDER BY population_growth DESC
LIMIT 10;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,death_birth_ratio
162,od,South Sudan,644329,None,None,12042910,4.02,36.91,8.18,11.47,0.22162015713898675
106,mi,Malawi,118484,94080,24404,17964697,3.32,41.56,8.41,0.0,0.20235803657362847
29,by,Burundi,27830,25680,2150,10742276,3.28,42.01,9.27,0.0,0.2206617472030469
128,ng,Niger,None,1266700,300,18045729,3.25,45.45,12.42,0.56,0.27326732673267323
182,ug,Uganda,241038,197100,43938,37101745,3.24,43.79,10.69,0.74,0.244119662023293
141,qa,Qatar,11586,11586,0,2194817,3.07,9.84,1.53,22.39,0.15548780487804878
27,uv,Burkina Faso,274200,273800,400,18931686,3.03,42.03,11.72,0.0,0.27884844158934097
109,ml,Mali,1240192,1220190,20002,16955536,2.98,44.99,12.89,2.26,0.2865081129139809
219,cw,Cook Islands,236,236,0,9838,2.95,14.33,8.03,None,0.5603628750872296
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62,0.11987281399046106


The majority of the countries here are in the African region, with some from the arabian regions also. This region is also considered to be third world, so not as much access to healthcare and other amenities you'd otherwise get easily in a first world country.

so are the death to birth rate ratios high? let's see what the maximum values are to answer that question

In [16]:
%%sql
SELECT *, death_rate/birth_rate AS death_birth_ratio
FROM facts
WHERE population_growth >
    (
        SELECT AVG(population_growth)
            FROM facts
            WHERE name NOT IN ('European Union', 'Antarctica', 'World')
    )
AND name NOT IN ('European Union','Antarctica', 'World')
ORDER BY death_birth_ratio DESC
LIMIT 10;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,death_birth_ratio
216,aa,Aruba,180,180,0,112162,1.33,12.56,8.18,8.92,0.6512738853503184
23,bc,Botswana,581730,566730,15000,2182719,1.21,20.96,13.39,4.56,0.6388358778625954
103,lu,Luxembourg,2586,2586,0,570252,2.13,11.37,7.24,17.16,0.6367634124890063
46,cy,Cyprus,9251,9241,10,1189197,1.43,11.41,6.62,9.48,0.5801928133216476
219,cw,Cook Islands,236,236,0,9838,2.95,14.33,8.03,None,0.5603628750872296
161,sf,South Africa,1219090,1214470,4620,53675563,1.33,20.75,9.91,2.42,0.47759036144578315
229,vi,British Virgin Islands,151,151,0,33454,2.32,10.91,4.99,17.28,0.4573785517873511
230,cj,Cayman Islands,264,264,0,56092,2.1,12.11,5.53,14.4,0.4566473988439307
76,ic,Iceland,103000,100250,2750,331918,1.21,13.91,6.28,4.43,0.45147375988497485
81,ei,Ireland,70273,68883,1390,4892305,1.25,14.84,6.48,4.09,0.4366576819407008


The majority of the countries here are considered first world countries, yet their death rate to birth ratio is higher. Why would this be the case? One reason could be the birth rates and population growth is significantly smaller than in the third world countries, where their values are as much as 3 to 4 times higher. These countries also have much smaller populations than those in the African region, which explains these statistics

## Conclusions

I have explored the CIA Factbook Data using SQL and revealed some interesting insights;

1. The top most populated countries lie in the Asian region
2. African countries are more likely to witness an increase in population relative to European countries
3. First world countries are experiencing relatively higher death rates compared to everyone else.

Of course we could have gone further. We could have compared the land to water mass ratios to perhaps gain insights on which countries are more likely to rely on fishing or other related industries. Overall this has beena  good demonstration of how SQL can help with data analysis